In [1]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [2]:
conf = Config()

conf.dataset = 'usa_large_cell'
conf.post_value_updates()

In [3]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# Load data.
test_df_1 = ddl.load_as_spark(schema="datascience_scratchpad", table="la_traj_data_v9_part1")
test_df_2 = ddl.load_as_spark(schema="datascience_scratchpad", table="la_traj_data_v9_part2")

:: loading settings :: url = jar:file:/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sagemaker-user/.ivy2/cache
The jars for the packages stored in: /home/sagemaker-user/.ivy2/jars
io.delta#delta-sharing-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-59ba57fc-1a15-4f54-9205-65199ebbffce;1.0
	confs: [default]
	found io.delta#delta-sharing-spark_2.12;3.2.0 in central
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-sharing-client_2.12;1.0.5 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-o

In [6]:
test_df= test_df_1.join(test_df_2, on  = ['userid','traj_date'])


In [7]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StructType, StructField, ArrayType, DoubleType, DateType, TimestampType, IntegerType
def get_weekday(traj_date):
    return traj_date.weekday()

import math
def lonlat2meters(lon, lat):
    semimajoraxis = 6378137.0
    east = lon * 0.017453292519943295
    north = lat * 0.017453292519943295
    t = math.sin(north)
    return semimajoraxis * east, 3189068.5 * math.log((1 + t) / (1 - t))
    
get_weekday_udf = udf(lambda traj_date: get_weekday(traj_date), IntegerType())
test_df = test_df.withColumn("weekday", get_weekday_udf(col("traj_date")))
lonlat2meters_udf = udf(lambda traj: [list(lonlat2meters(p[0], p[1])) for p in traj], ArrayType(ArrayType(DoubleType())))
test_df = test_df.withColumn("merc_seq", lonlat2meters_udf(col("polylines_final")))
display(test_df)

DataFrame[userid: int, traj_date: date, timestamps: array<timestamp>, polylines_final: array<array<double>>, employername: string, partition_id: int, pck_0_1: int, weekday: int, merc_seq: array<array<double>>]

In [8]:
test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/la/la_subset_1k")

25/09/11 15:19:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/la/la_subset_1k/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

Total records in repartitioned files: 39735


In [5]:
df_list[0].head()

,userid,traj_date,timestamps,polylines_final,employername,partition_id,pck_0_1,weekday,merc_seq,time_index_list
0,2844305,2025-06-24,"[2025-06-24T05:49:28.000000000, 2025-06-24T07:...","[[-118.39117477618277, 33.92325761467734], [-1...",Camping World/Gander Outdoors,2,1,1,"[[-13179245.29050212, 4018502.0477052485], [-1...","[34, 44, 45, 46, 47, 49, 50, 53, 55, 56, 57, 5..."
1,2869661,2024-09-21,"[2024-09-21T08:54:19.000000000, 2024-09-21T11:...","[[-118.59423330634597, 34.1905517578125], [-11...",County Of Riverside,0,1,5,"[[-13201849.662681118, 4054417.2603058773], [-...","[53, 71, 74, 85, 92, 110, 111, 112, 113, 126, ..."
2,2869661,2024-09-22,"[2024-09-22T07:02:57.000000000, 2024-09-22T11:...","[[-118.5940859861017, 34.190643310546875], [-1...",County Of Riverside,0,1,6,"[[-13201833.263066543, 4054429.58131514], [-13...","[42, 68, 69, 70, 71, 72, 73, 91, 92, 93, 96, 9..."
3,2869661,2025-02-08,"[2025-02-08T10:03:51.000000000, 2025-02-08T10:...","[[-117.66765060248021, 34.03045269170906], [-1...",County Of Riverside,1,1,5,"[[-13098702.947908927, 4032891.811937815], [-1...","[60, 61, 62, 64, 65, 66, 75, 76, 79, 81, 82, 8..."
4,2869661,2025-04-26,"[2025-04-26T09:22:08.000000000, 2025-04-26T09:...","[[-117.73360870444878, 34.025118917335035], [-...",County Of Riverside,1,1,5,"[[-13106045.37023376, 4032175.3819222637], [-1...","[56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 86, 8..."


In [6]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/la_nyc_emb_add_without_time/usa_large_cell_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_6085/4088849310.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTB(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): SpatialMSM(


In [10]:
from utils.traj import *
import pickle
from utils.cellspace import *

from torch.nn.utils.rnn import pad_sequence
embs_parent = pickle.load(open(Config.dataset_embs_file_parent, 'rb')).to('cpu').detach() # tensor
embs_child = pickle.load(open(Config.dataset_embs_file_child, 'rb')).to('cpu').detach() # tensor
cellspace_parent = pickle.load(open(Config.dataset_cell_file_parent, 'rb'))
cellspace_child = pickle.load(open(Config.dataset_cell_file_child, 'rb'))
hier_cellspace = HirearchicalCellSpace(cellspace_parent, cellspace_child)

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell_parent, traj_cell_child, traj_p = zip(*[merc2cell2(t, hier_cellspace) for t in traj])
    # print(traj_cell)
    traj_emb_p = [torch.tensor(generate_spatial_features(t, hier_cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell_parent = [embs_parent[list(t)] for t in traj_cell_parent]
    traj_emb_cell_child = [embs_child[list(t)] for t in traj_cell_child]
    traj_emb_cell = [a + b for a, b in zip(traj_emb_cell_parent, traj_emb_cell_child)]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_p)), dtype = torch.long, device = device)
    time_indices = pad_sequence([torch.tensor(t, dtype=torch.long) for t in time_indices], batch_first=False, padding_value=-1).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, time_indices)
    return traj_embs

def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size] 
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

Total unique userids: 1000
Total unique employernames: 1007


In [9]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_0_1']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['time_index_list'].values
    return traj, time_indices

def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [11]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    count = 0
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                continue
            else:
                # print(userid, employer, partition)
                # print(len(part_data))
                traj, time_indices = get_traj_and_time_data(part_data)
                # print(traj)
                embs = infer(traj, time_indices).detach().cpu().numpy()
                db_scan = apply_dbscan(embs, target_min_similarity=0.85)
                labels = db_scan.labels_                      # shape: (n_samples,)
                cluster_ids = [c for c in np.unique(labels) if c != -1]
                if len(cluster_ids)>0:
                    pred_label = 1
                else:
                    pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    return count
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label
count=0
for userid in tqdm(unique_userids):
    count+=get_gt_and_pred_label(userid)


        


100%|██████████| 1000/1000 [00:28<00:00, 35.00it/s]


In [12]:
count_dict = {}
for k,v in output_dict.items():
    if v in count_dict:
        count_dict[v]+=1
    else:
        count_dict[v]=1
count_dict

{0: 190, 1: 253}

In [13]:
output_dict

{(9093953, 'PepsiCo', 1): 0,
 (13867356, 'lake park communities', 0): 1,
 (13399849, 'Marriott international', 0): 0,
 (13399849, 'Marriott international', 1): 0,
 (9210700, 'Walt Disney PA', 0): 1,
 (9377174, 'COUNTY HEALTH SERVICES', 0): 0,
 (9377174, 'Maxim healthcare', 0): 0,
 (2157516, 'Amazon', 0): 0,
 (14203499, 'Curcuit services llc', 0): 1,
 (14203499, 'Industrial electronic engineers', 0): 1,
 (14475344, 'SCHARP', 0): 1,
 (10658382, '76 Gas', 0): 1,
 (15705160, 'QVC Distribution Center', 0): 0,
 (3898274, 'Kiva sales and services', 0): 0,
 (7209893, 'relativity space', 0): 0,
 (7209893, 'relativity space', 1): 0,
 (12192241, "BJ'S RESTA", 0): 1,
 (15933208, 'The gill corporation', 1): 1,
 (13342539, 'City of Los Angeles', 0): 0,
 (16690599, 'Colodro-garo llc', 0): 0,
 (16690599, 'Colodro-garo llc', 1): 0,
 (14359486, 'huntintong hospital', 0): 0,
 (14359486, 'huntintong hospital', 1): 0,
 (4833188, 'Target', 0): 0,
 (4833188, 'Target', 1): 0,
 (18617609, 'The Walt Disney Comp